git@github.com:Mael-Le-Garrec/PhD.git# Hamiltonian for a Magnet Kick

This is the Hamiltonian for a magnetic element with normal strength component _K_ and skew component _J_:
$$\mathcal{H} = \Re \left[\sum_{n>1} \left( K_{n-1} + iJ_{n-1} \right) \frac{(x+iy)^n}{n!} \right]$$

<div class="alert alert-block alert-warning">
<b>Remark</b>: When writing things like $K_1$ it is assumed it is $K_1(s)$. Is is important later for the chromaticity where the average is a circular integral over s.
</div>

## Derivation

Starting from Eq. 2.28 of [Felix Carlier's Thesis]
$$\mathcal{H} = \frac{1}{2}(p^2_x + p^2_y) - \frac{A_s(x,y)}{B_0 \rho}$$

---
And the expression of the magnetic field

$$B_x = -\frac{\partial V}{\partial x} = \frac{\partial A_s}{\partial y}, 
  B_y = -\frac{\partial V}{\partial y} = \frac{\partial A_s}{\partial x}$$

$$\mathcal{A}(x + iy) = A_s(x,y) + iV(x,y)$$

We can then express $\mathcal{A}(x+iy)$ with:
$$-\frac{\partial}{\partial x} \mathcal{A}(x + iy) = \sum^\infty_{n=2}\left[K_{n}(s) + iJ_{n}(s)\right](x+iy)^{n}$$

Where $K_{n}$ and $J_{n}$ are defined as:

$$K_{n}(s) = \frac{1}{B_0 \rho} \frac{\partial^{n}B_y}{\partial x^{n}},
  J_{n}(s) = \frac{1}{B_0 \rho} \frac{\partial^{n}B_x}{\partial x^{n}}$$


[Felix Carlier's Thesis]: https://cds.cern.ch/record/2715765
[Ewen's Thesis]: https://cds.cern.ch/record/1951379/files/Thesis-2014-Ewen
[Second Order Dispersion measurement in LHC]: https://cds.cern.ch/record/2696124/files/mopmp027.pdf
[Particle Accelerators Physics II]: https://link.springer.com/book/10.1007/978-3-642-97550-9

---

By taking the primitive of $\sum^\infty_{n=2}\left[K_{n}(s) + iJ_{n}(s)\right](x+iy)^{n}$ and then the real part, we can get $A_s(x,y) / B_0\rho$

---

As we assume the elements are thin, we can omit the $\frac{1}{2}(p^2_x + p^2_y)$ part in the Hamiltonian.

---

# Chromaticity

The chromaticity is given by Eq.(5.5) of [Ewen's Thesis]:

$$Q'_{x,y} = \frac{\partial Q_{x,y}}{\partial \delta} = \frac{1}{2\pi}\left< \frac{\partial²\mathcal{H}}{\partial J_{x,y}\partial \delta}\right>$$

For a single element, we can compute $ΔQ'$:
 $$\Delta Q_{x,y}' = \frac{1}{2\pi} \int_L \frac{\partial \left< H \right>}{\partial J_{x,y} \partial \delta} ds$$

Reminder:

$$\xi = \frac{\Delta Q / Q}{\Delta p/p}$$


# Dispersion

The orbit change $Δx$ can be expressed with different orders of the dispersion, inducing higher order relative momentum offsets.  
Found in J.Keintzel's [Second Order Dispersion measurement in LHC] paper and [Particle Accelerators Physics II] (Eq 4.78) by H.Wiedemann.

* First order:
    $$Δ_{z} = \eta_{z,1} \delta p \Longleftrightarrow \eta_{z} = \eta_{z,0} + \eta_{z,1} \delta p $$
* Second order:
    $$Δ_{z} = \eta_{z,1} \delta p + \eta_{z,2} \delta p^2 \Longleftrightarrow \eta_{z} = \eta_{z,0} + \eta_{z,1} \delta p + \eta_{z,2} \delta p^2 $$
* Third order:
    $$Δ_{z} = \eta_{z,1} \delta p + \eta_{z,2} \delta p^2 + \eta_{z,3} \delta p^3 \Longleftrightarrow \eta_{z} = \eta_{z,0} + \eta_{z,1} \delta p + \eta_{z,2} \delta p^2 + \eta_{z,3} \delta p^3$$
    
With $z$ the coordinate being either $x$ or $y$ and $\eta_{1,2,3}$ refering to the first, second and third order dispersion respectively.

In [1]:
from IPython.display import display, Latex, Math, Markdown, HTML
from sympy import re, Sum, I, factorial, Indexed, simplify, expand_complex, symbols, Function, diff, cos, sqrt, pi, integrate, collect, apart, factor
from sympy import init_printing, latex
init_printing(use_latex=True)

In [16]:
max_order = 10  # Maximum multipole order, used to create the symbols
space = '\;' * 8  # for spacing latex

def pm(s):
    display(Markdown(s))
    
    
def taylor(f, order, x):
    ''' Taylor expansion of `order`, of function f for the variable x, at the point 0'''
    base = f.subs(x, 0)
    for i in range(1, order+1):
        tmp = diff(f, x, i).subs(x, 0)
        tmp /= factorial(i)
        tmp *= x ** i
        base += tmp
    return base


def chromaticity(chroma_order, hamiltonian, z, J, K, beta, phix, phiy, dpp):
    L = symbols('L')  # length of the magnet
    
    # Differentiate the hamiltonian to get the chroma
    q_prime = diff(hamiltonian, J)
    q_prime = diff(q_prime, dpp, chroma_order)
    
    # Define the str sequences so it's prettier to print
    start_str = " " if chroma_order == 0 else ''
    expo_h = f'{chroma_order + 1}' # exponent for the \partial H
    expo_dpp = f'^{chroma_order}' if chroma_order > 1 else '' # exponent for the \partial \delta p
    chroma_prime = f'^{{({chroma_order})}}' if chroma_order > 0 else ''
    dpp_str = r"\partial" + expo_dpp + r"\delta" if chroma_order > 0 else ''
    
    # Display the title of the derivation
    display(Markdown(r"* Compute $" + " \Delta Q" + chroma_prime + "$ for the plane " + latex(z) + ":"))
    q_prime_str = latex(q_prime.simplify())
    
    # Integrate over the phase variables phix and phiy to get the mean of the cosines
    average_q_prime = (1/(2*pi)) * integrate(integrate(q_prime, (phix, 0, pi))/pi, (phiy, 0, pi))/pi
    average_q_prime_str = latex(average_q_prime.simplify())
    
    # Multiply directly by L, like if we integrated over ds
    result_q_prime = average_q_prime * L
    result_q_prime_str = latex(result_q_prime.simplify())
    
    # if we still have a δ in there, it means it's a higher order chromaticity
    end_str = ""
    if dpp in result_q_prime.free_symbols:
        end_str = r" \Longrightarrow Q" + chroma_prime + "_" + latex(z) + " = 0"
        
    display(Math(space + start_str + r"\Delta Q" + chroma_prime + r"_" + latex(z) + r" = \frac{1}{2\pi} \left< \frac{\partial^" + expo_h + r" H}{\partial J_" + latex(z) + dpp_str + r"} \right> = \int_L ds \frac{1}{2\pi} \left< " + q_prime_str + r"\right> = \int_L ds " + average_q_prime_str + " = " + result_q_prime_str + end_str))

    
def get_dispersion(plane, dpp, order):
    """
    Returns the dispersion equation depending on the order
    
    Args:
      - plane: 'x' or 'y'
      - dpp: dpp symbol
      - order: int >= 0
    """
    res = 0
    for exp in range(1, order+1):
        eta = symbols(f'eta_{plane}_{exp}')
        res += eta * dpp ** exp
    return res
        

def hamiltonian(order, K, J, x, y, chroma, taylor_exp=False, disp_order=1):
    # Define our symbols for Δx
    eta_x1, eta_x2, eta_x3 = symbols('eta_x_\,_1 eta_x_\,_2 eta_x_\,_3')
    dpp = symbols('delta_p')
    P, P0 = symbols('P P_0')
    q = symbols('q')
    
    # Here Δx = ηₓ₁.δₚ
    dx = get_dispersion('x', dpp, disp_order)
    display(Math("\Delta x = " + latex(dx)))
    #dx = eta_x1 * dpp + eta_x2 * dpp**2
    
    # Compute the full hamiltonian
    res = 0
    for n in range(2, order+1):
        s = (K[n] + I * J[n]) * (((x + I * y) ** n)/factorial(n))
        res += s
    # And keep the real part only
    res = re(res)
    
    # Keep only the main field
    main = res
    for n in range(2, order+1):
        main = main.subs(J[n], 0)
        main = main.subs(K[n-1], 0)
        
    # Substitute K for q / P (n-1)! B_n with P = P_0(1 + δ)
    # Replace (1 + δ)**-1 by a taylor expansion
    if taylor_exp:
        inv_dpp = (1 + dpp) ** -1
        dpp_exp = taylor(inv_dpp, 1, dpp)
        display(Markdown('Performing a Taylor Expansion for $' + latex(1/(1+dpp)) + '$: $' + latex(dpp_exp) + '$'))
        main = main.subs(K[n], q / P0 * (dpp_exp) * factorial(n-1) * B[n])
    
    # Print both equations
    display(Markdown('#### Hamiltonian'))
    main_str = f'\mathcal{{N^{order}}}(x, y) =' + latex(main.simplify())    
    str_res = f'\mathcal{{H^{order}}}(x, y) =' + latex(res.simplify())
    display(Markdown('* Raw Expression:'), Math(space + str_res))
    display(Markdown('* By keeping only the main _normal_ field:'), Math(space + main_str))
    
    # Perform a variabla change for x, to account for Δx
    expanded = main.subs(x, x + dx)
    expanded_str = f'\mathcal{{N^{order}}}(x + Δx, y) =' + latex(expanded)
    display(Markdown('* With a displacement in _x_:'), Math(space + expanded_str))
    
    # Perform a variable change: x = √(2Jₓβₓ)cosφₓ ; y = √(2Jᵧβᵧ)cosφᵧ
    Jx, Jy = symbols('J_x J_y', real=True)
    phix, phiy = symbols('phi_x phi_y', real=True)
    betx, bety = symbols('beta_x beta_y', real=True)
    new_x = (sqrt(2 * Jx * betx) * cos(phix)).expand()
    new_y = sqrt(2 * Jy * bety) * cos(phiy)
    new_vars = expanded.subs(x, new_x).subs(y, new_y)
    new_str = f'\mathcal{{N^{order}}}(x + Δx, y) =' + latex(new_vars)
    display(Markdown(f'* Coordinate variables substitued for $x = {latex(new_x)}$ and $y = {latex(new_y)}$:'), 
            Math(space + new_str))
    
    # Differentiate relative to the action to get the detuning
    #display(Markdown("#### Detuning ΔQ"))
    #chromaticity(0, new_vars, x, Jx, K, betx, phix, phiy, dpp)
    #chromaticity(0, new_vars, y, Jy, K, bety, phix, phiy, dpp)
    
    # Differentiate relative to the momentum and the action to get the chromaticity
    # Display every order that have been requested
    for o in chroma:
        display(Markdown(f"#### Chroma $Q^{{({o})}}$"))
        chromaticity(o, new_vars, x, Jx, K, betx, phix, phiy, dpp)
        chromaticity(o, new_vars, y, Jy, K, bety, phix, phiy, dpp)
    
    display(HTML("<br>" * 3))

    return res, main, new_vars

    
# Create and array of K and J values
# Those are the strengths of the regular and skew magnets
K, J, B = [], [], []
x, y = symbols('x y', real=True)

for i in range(max_order):
    K.append(symbols(f'K{i}', real=True))
    J.append(symbols(f'J{i}', real=True))
    B.append(symbols(f'B{i}', real=True))

    
pm("## Hamiltonians")
pm("The Hamiltonians below include the normal and skew components.  \
It is assumed here than the elements are thin.")
pm("Elements should only have a main field, thus the lower K values can be ignored.")
pm("$$" + space + r"Q' = \sum_{magnets} \Delta Q'$$")

#pm("### Quadropole")
#quadrupole, main_quad, action_vars_quad = hamiltonian(2, K, J, x, y, chroma=[1], taylor_exp=True)

pm("### Sextupole")
sextupole, main_sext, action_vars_sext = hamiltonian(3, K, J, x, y, chroma=[1, 2], disp_order=1)
    
#pm("### Octupole")
#octupole, main_oct, action_vars_oct = hamiltonian(4, K, J, x, y, chroma=[4])

#pm("### Decapole")
#decapole, main_deca, action_vars_deca = hamiltonian(5, K, J, x, y, chroma=[3]) 

#pm("### Dodecapole")
#dodeca, main_dodeca, action_vars_dodeca = hamiltonian(6, K, J, x, y, chroma=[4])

#pm("### Tetradecapole")
#tetradeca, main_tetradeca, action_vars_tetradeca = hamiltonian(7, K, J, x, y, chroma=[5])

## Hamiltonians

The Hamiltonians below include the normal and skew components.  It is assumed here than the elements are thin.

Elements should only have a main field, thus the lower K values can be ignored.

$$\;\;\;\;\;\;\;\;Q' = \sum_{magnets} \Delta Q'$$

### Sextupole

...

#### Chroma $Q^{(2)}$

* Compute $ \Delta Q^{(2)}$ for the plane x:

<IPython.core.display.Math object>

* Compute $ \Delta Q^{(2)}$ for the plane y:

<IPython.core.display.Math object>

In [3]:
k6 = 1000000
L = 0.615
b_x = 102.4393606
dx = 0.001648132608

(k6  * L * b_x * dx**4) / (4 * 3.14159)